# Multi-objective Optimization Streamlit Application
- Author: senkin.zhan@datarobot.com

## Summary
This accelerator introduces an approach to build a streamlit application using DataRobot deploments for multi-objective optimization.

This notebook outlines how to:

1. Create multiple projects 
2. Build deployment with best model
3. Build Streamlit application connecting to DataRobot


## Setup

### Install and import libraries

In [2]:
!pip install git+https://github.com/datarobot/dr-apps 

  Cloning https://github.com/datarobot/dr-apps to /tmp/pip-req-build-mtve5ld3
  Installing build dependencies ... - \

 | / done
  Getting requirements to build wheel ... - done
    Preparing wheel metadata ... - done
  Created wheel for drapps: filename=drapps-0.2.4-py3-none-any.whl size=20780 sha256=0736f9d39dc57e313606fa8aba6e0818e1657254357ffec2db8cc5b5e58b6e78
  Stored in directory: /tmp/pip-ephem-wheel-cache-lp_rp4lg/wheels/dc/ed/e4/791a4ddf6aeb3c90f6484ad2b414014b4df60c168c8045943e

- \ done
  Created wheel for bson: filename=bson-0.5.10-py3-none-any.whl size=11978 sha256=f5567a09ad780f813bce00b7baaa3f5a28ef06fea4e0233a7b3e28c2b0dc3675
  Stored in directory: /tmp/pip-ephem-wheel-cache-lp_rp4lg/wheels/6d/8b/5e/68619cb02962978c6cc8eaf68ca23ef28c69de7e086740702b
Successfully built drapps bson


In [ ]:
import pandas as pd
import numpy as np
import datarobot as dr
from datarobot.models.dataset import Dataset
import datetime as dt
import time
import os
from dask import delayed, compute #For parallelization
import pickle

### Download code from github

In [4]:
!git clone https://github.com/senkin13/multi-objective-optimization.git

Cloning into 'multi-objective-optimization'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 39.32 KiB | 39.32 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [6]:
!ls -lt

total 4
drwxr-xr-x 3 notebooks notebooks  178 Jun 14 08:08 multi-objective-optimization
drwxrwsr-x 3 root      notebooks 4096 Jun  6 09:20 storage


### Bind variables

In [ ]:
# If use groupkfold input group column ["Group_ID"], if use randomkfold input False
group_col = False

# Datarobot Path
input_path = './multi-objective-optimization/'

# File name, default is japanese version, if use english input steel_strength_en
file_name = 'steel_strength_jp.csv'

# Targets(multiple objective), default is japanese version, if use english input ['Yield strength','Tensile strength','Average strength']
targets = ['降伏強度','引張強度','平均強度'] 

### Run autopilot on datarobot

In [8]:
%%time

def __run_autopilot(target,df,group_col):

    project = dr.Project.create(df, project_name=target)

    ao = dr.AdvancedOptions()
    if (group_col!=False):
        pm = dr.GroupCV(holdout_pct=0, partition_key_cols=[group_col], reps=5)            
    else:
        pm = dr.RandomCV(holdout_pct=0, reps=5)

    project.analyze_and_model(worker_count=-1,
                                  target=target, 
                                  mode="quick", 
                                  partitioning_method=pm, 
                                  advanced_options=ao, 
                                  max_wait=10000)
    
    return project

def __wait_for_cv(projects):
    for i in range(len(projects)):
        try:
            project = dr.Project.get(project_id=projects[i-1].id)
            jobs_list = project.get_all_jobs()
            for job in jobs_list:
                job.wait_for_completion(max_wait=60000)
            print("Project " + targets[i] + " completed running autopilot")
        except:
            print("Project " + targets[i] +" occured error")

df = pd.read_csv(input_path + file_name)
df_feature = df.drop(targets,axis=1)

delayed_dr_projects = []
for i in range(len(targets)):  
    df_ = df_feature.copy()
    df_[targets[i]] = df[targets[i]]
    temp = delayed(__run_autopilot)(targets[i],df_,group_col)
    delayed_dr_projects.append(temp)
        
projects = compute(delayed_dr_projects)[0]

__wait_for_cv(projects)

Project 降伏強度 completed running autopilot


Project 引張強度 completed running autopilot


Project 平均強度 completed running autopilot
CPU times: user 1.81 s, sys: 524 ms, total: 2.34 s
Wall time: 4min 34s


### Run delopyment on datarobot

In [9]:
%%time

def __run_deployment(project):
    model = project.get_models()[0]
    target = project.target
    prediction_server = dr.PredictionServer.list()[0]

    registered_model_version = dr.RegisteredModelVersion.create_for_leaderboard_item(
        model_id=model.id,
        name=target,
    )

    deployment = dr.Deployment.create_from_registered_model_version(
        registered_model_version.id, label=target, 
        default_prediction_server_id=prediction_server.id)
    
    return deployment
    
delayed_dr_projects = []
for i in range(len(projects)):  
    temp = delayed(__run_deployment)(projects[i])
    delayed_dr_projects.append(temp)
        
deployments = compute(delayed_dr_projects)[0]

CPU times: user 735 ms, sys: 62.8 ms, total: 798 ms
Wall time: 4min 18s


### Show deployment id

In [10]:
for d in deployments:
    print (d,d.id)

Deployment(降伏強度) 666bfc08fe9199e909e1c0a0
Deployment(引張強度) 666bfc0922f06fd37b9ff8ff
Deployment(平均強度) 666bfcb06c8ecda67b9fe99e


### Build streamlit app on datarobot

In [11]:
%%time
API_URL = os.environ['API_URL']
DATAROBOT_API_TOKEN = os.environ['DATAROBOT_API_TOKEN']
DATAROBOT_KEY = os.environ['DATAROBOT_KEY']
credentials = [API_URL,DATAROBOT_API_TOKEN,DATAROBOT_KEY]
with open(input_path+'credentials.pickle', mode='wb') as fo:
    pickle.dump(credentials, fo)
    
!drapps create -t $DATAROBOT_API_TOKEN -e "[Experimental] Python 3.9 Streamlit" -p $input_path  "multi-objective-optimization-demo-senkin"

Using multi-objective-optimization-demo-senkinSource custom application source.
Creating new version for multi-objective-optimization-demo-senkinSource custom application source.
Uploading project:  [------------------------------------]    0%

Uploading project:  [####################################]  100%
Starting multi-objective-optimization-demo-senkin custom application.
Waiting till app is ready:  [------------------------------------]

Waiting till app is ready:  [-------------------------#----------]

Waiting till app is ready:  [-#----------------------------------]

Waiting till app is ready:  [---------------------------------#--]

...

Waiting till app is ready:  [---------#--------------------------]

Waiting till app is ready:  [#-----------------------------------]

Waiting till app is ready:  [------------#-----------------------]

Waiting till app is ready:  [-------------------------------#----]


Custom application is running: https://app.datarobot.com/custom_applications/666bfd61f9d5bfa0ae8382c4/
CPU times: user 2.64 s, sys: 1.28 s, total: 3.93 s
Wall time: 3min 43s
